In [1]:
from dataset.utils import pNormalize, classCount
from dataset.datasets import sentinel
from model.models import UNET
from torch.utils.data import DataLoader
from train.metrics import computeConfMats, computeClassMetrics, wma, printClassMetrics, printModelMetrics, plotConfusionMatrices, plotConfusionMatrix
import torch

q_hi = torch.tensor([2102.0, 1716.0, 1398.0, 4732.0, 2434.42919921875, 3701.759765625, 4519.2177734375, 4857.7734375, 3799.80322265625, 3008.8935546875])
q_lo = torch.tensor([102.0, 159.0, 107.0, 77.0, 106.98081970214844, 79.00384521484375, 86.18966674804688, 70.40167236328125, 50.571197509765625, 36.95356750488281])    
norm = pNormalize(maxPer=q_hi,minPer=q_lo)

# Hyperparams
BATCH_SIZE = 4
NUM_WORKERS = 0

# Create experimental dataset, rgb=True for 3 channels (default = False)
# POINT TO FOLDER WITH TIMEPERIOD(S) WITH SUBFOLDERS: 'test, 'train, 'val
test_set = sentinel(root_dir='./', img_transform=norm,data="test")

# Pass in the dataset into DataLoader to create an iterable over the dataset
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

# Define model
dataiter = iter(test_loader)
images, labels = dataiter.next()
model = UNET(in_channels=images.shape[1],classes=28)
model.load_state_dict(torch.load('trained_model.pt',map_location=torch.device('cpu'))) # Pass in 'trained_model.pt' and load model

# 1 EPOCH TESTING
dataiter = iter(test_loader) # Create an object which can be iterated one element at a time
model.eval() # TOGGLE ON EVALUATION MODE
with torch.no_grad():
     # n_class - unclassified class, i.e. 28-1 = 27
     cMats = torch.zeros((27,2,2),dtype=torch.int32)
     class_metrics = torch.zeros((27,6),dtype=torch.float32)
     
     model_metrics = torch.zeros(6,dtype=torch.float32)
         
     predarr = torch.tensor([],dtype=torch.int32)
     labelarr = torch.tensor([],dtype=torch.int32)
     
     for images, labels in dataiter:
          outputs = model(images)
          preds = torch.nn.functional.softmax(outputs,dim=1)
          preds = torch.argmax(preds,dim=1)
          cMats += computeConfMats(labels,preds)
          
          # Flatten dimensions BxHxW --> B*H*W and concatenate
          predarr = torch.cat((predarr, preds.reshape(-1)))
          labelarr = torch.cat((labelarr, labels.reshape(-1)))
#model.train() # TOGGLE ON TRAIN MODE WHEN EVALUATION IS DONE

In [2]:
# Get class counts for dataset
classCounts,_ = classCount(test_loader)

In [ ]:
# Compute class and model metrics
class_metrics = computeClassMetrics(cMats)
model_metrics = wma(class_metrics,classCounts)

In [ ]:
# Print model metrics
printModelMetrics(model_metrics)

In [ ]:
# Print class Metrics
printClassMetrics(class_metrics,classCounts)

In [ ]:
# Plot N_CLASS X N_CLASS confusion matrix
plotConfusionMatrix(yTrue=labelarr,yPred=predarr)

In [ ]:
# Plot Confusion Matrices
plotConfusionMatrices(cMats)

In [8]:
# Print model metrics to TENSORBOARD. Default path = 'runs/Model_Metrics'
printModelMetrics(model_metrics,TB=True)

In [9]:
# Print class metrics to TENSORBOARD. Default path = 'runs/Class_metrics'
printClassMetrics(class_metrics,classCounts,TB=True)

In [10]:
# Plot N_CLASS X N_CLASS confusion matrix to TENSORBOARD. Default path = 'runs/Confusion_Matrix'
plotConfusionMatrix(yTrue=labelarr,yPred=predarr,TB=True)

In [11]:
# Plot confusion matrices to TENSORBOARD. Default path = 'runs/Confusion_Matrices'
plotConfusionMatrices(cMats,TB=True)